#  Setup stuff: Connectivity

In [7]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [8]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [14]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

<_Graph my_graph, 5bjwBnHtxvvetB2QNrzFmNNgPbz1AorCzznQkQXbSDPM, 2>


#  Setup stuff: Debugging 

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [13]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]

#  Actual analytics ..

In [11]:

from katana import remote
from katana.remote import analytics


l_cntr = 0

print("--")


--


In [ ]:

#  Running analytics.sssp() ..


l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)

l_weight_property     = "DISTANCE"                 #  Edge property

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


          0/? [?op/s]

In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   // RETURN r.type, r.TYPE, r.label, r.start.labels, r.end.labels, r.internal_id, r.start.id, r.end.id, r.DISTANCE, r.NUM_HOPS
   """ )
l_result.view()


#  This was from a version before 0.5; can'r recall if it was 0.2 or 0.3
#
#  Graph loaded, but before running sssp() 
#     (This is correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge
#  3            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge
#
#  1            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge

#  Graph loaded, after running sssp() 
#     (This is not correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge         #  This data is now wrong
#  1            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge         #  This data is now wrong
#
#  3            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge         #  This data is now wrong


#  Now version 0.5 ..
#
#  Graph loaded, but before running sssp() 
#     (This is correct)
#
#  r.end.id           r.end.labels   r.internal_id    r.label    r.DISTANCE    r.NUM_HOPS    r.TYPE      r.start.id        r.start.labels    r.type
#  562949953421313    ['Airport']    0                FLIES_TO    948          1             FLIES_TO    1                 ['Airport']       edge
#  562949953421312    ['Airport']    2                FLIES_TO    886          1             FLIES_TO    1                 ['Airport'] 	     edge
#  562949953421312    ['Airport']    1                FLIES_TO     66          1             FLIES_TO    0                 ['Airport'] 	     edge
#  562949953421312    ['Airport']    562949953421312  FLIES_TO   1829          1             FLIES_TO    562949953421313   ['Airport'] 	     edge
#  1                  ['Airport']    562949953421314  FLIES_TO    948          1             FLIES_TO    562949953421313   ['Airport'] 	     edge
#  1                  ['Airport']    562949953421313  FLIES_TO    886          1             FLIES_TO    562949953421312   ['Airport'] 	     edge
#  0                  ['Airport']    562949953421315  FLIES_TO     66          1             FLIES_TO    562949953421312   ['Airport'] 	     edge

#  Graph loaded, after running sssp() 
#
#  r.end.id           r.end.labels   r.internal_id    r.label    r.DISTANCE    r.NUM_HOPS    r.TYPE      r.start.id          r.start.labels    r.type
#  2                  ['Airport']    3                FLIES_TO     66          1             FLIES_TO    0                   ['Airport']       edge
#  1                  ['Airport']    2 	              FLIES_TO    948          1             FLIES_TO    3                   ['Airport']       edge
#  2                  ['Airport']    5 	              FLIES_TO   1829          1             FLIES_TO    3                   ['Airport']       edge
#  2                  ['Airport']    4 	              FLIES_TO    886          1             FLIES_TO    1                   ['Airport']       edge
#  3                  ['Airport']    6 	              FLIES_TO    948          1             FLIES_TO    1                   ['Airport']       edge
#  0                  ['Airport']    0 	              FLIES_TO     66          1             FLIES_TO    2                   ['Airport']       edge
#  1                  ['Airport']    1                FLIES_TO    886          1             FLIES_TO    2                   ['Airport']       edge

